In [22]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
print(tf.__version__)

1.3.0


In [24]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [25]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [26]:
EMBEDDING_SIZE=32
HIDDEN_SIZE=128
BATCH_SIZE=64
NUM_STEPS=8
NUM_EPOCHS_INIT_LR=2
NUM_EPOCHS_TOTAL=6
INITIAL_LR=1e1
LR_DECAY_RATE=0.75
MAX_NORM=0.25

In [28]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
training_nodes = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [29]:
training_outputs = {**summary_nodes, **training_nodes}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 64 == 63):
                print('step: {0}    loss: {1}    gradient norm: {2}     correct words: {3}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['gradient_global_norm'],
                    outputs['num_correct_predictions'],
                ))
                
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('validation perplexity:', np.exp(total_loss / total_batches))
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('test perplexity:', np.exp(total_loss / total_batches))

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 64    loss: 6.734375    gradient norm: 0.2236328125     correct words: 25
step: 128    loss: 6.703125    gradient norm: 0.2305908203125     correct words: 33
step: 192    loss: 6.453125    gradient norm: 0.2344970703125     correct words: 44
step: 256    loss: 6.390625    gradient norm: 0.25537109375     correct words: 42
step: 320    loss: 6.20703125    gradient norm: 0.255859375     correct words: 77
step: 384    loss: 6.2890625    gradient norm: 0.272216796875     correct words: 49
step: 448    loss: 6.1640625    gradient norm: 0.2352294921875     correct words: 75
step: 512    loss: 5.8125    gradient norm: 0.265380859375     correct words: 66
step: 576    loss: 5.7734375    gradient norm: 0.382568359375     correct words: 72
step: 640    loss: 5.6875    gradient norm: 0.265869140625     correct words: 80
step: 704    loss: 5.83984375    gradient norm: 0.28955078125     correct words: 84
step: 768    loss: 5.90625    gradient norm: 0.375732421875     correct words: 73
step: 8

step: 832    loss: 4.8359375    gradient norm: 0.3642578125     correct words: 110
step: 896    loss: 4.6640625    gradient norm: 0.351806640625     correct words: 121
step: 960    loss: 4.66015625    gradient norm: 0.36572265625     correct words: 117
step: 1024    loss: 4.984375    gradient norm: 0.36083984375     correct words: 115
step: 1088    loss: 4.98828125    gradient norm: 0.36767578125     correct words: 104
step: 1152    loss: 4.6796875    gradient norm: 0.357421875     correct words: 117
step: 1216    loss: 4.7734375    gradient norm: 0.361328125     correct words: 119
step: 1280    loss: 4.95703125    gradient norm: 0.35400390625     correct words: 106
step: 1344    loss: 4.8125    gradient norm: 0.361328125     correct words: 114
step: 1408    loss: 4.4765625    gradient norm: 0.355712890625     correct words: 129
step: 1472    loss: 4.73828125    gradient norm: 0.351806640625     correct words: 115
step: 1536    loss: 4.8671875    gradient norm: 0.3642578125     correct